<a href="https://colab.research.google.com/github/innocentmatutu/Machine-learning/blob/main/Medical_costs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,r2_score,root_mean_squared_error
import xgboost as xgb

df = pd.read_csv("/content/archive (8).zip")

#Encoding of categorical data
for col in df.select_dtypes(include=['object']).columns:
   encoder = LabelEncoder()
   df[col] = encoder.fit_transform(df[col])

#Filling missing values with the mean along each column--formality
def impute_with_extension(X_train,X_valid):
  cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
  X_train_plus = X_train.copy()
  X_valid_plus = X_valid.copy()

  for col in cols_with_missing:
    X_train_plus[col +'_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col +'_was_missing'] = X_train_plus[col].isnull()

  impute = SimpleImputer(strategy='mean')
  imputed_X_train = pd.DataFrame(impute.fit_transform(X_train_plus),columns = X_train_plus.columns)
  imputed_X_valid = pd.DataFrame(impute.transform(X_valid_plus),columns = X_valid_plus.columns)

  return imputed_X_train, imputed_X_valid

#Feature and target selection
features = ['age','sex','bmi','children','smoker','region']
X = df[features]
y = df['charges']
df.drop(['charges'],axis=1,inplace=True)

#Train, test and split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,random_state=1)

imputed_X_train, imputed_X_valid = impute_with_extension(X_train,X_valid)

#Model selection,training and prediction
model = xgb.XGBRegressor()
model.fit(imputed_X_train,y_train)
prediction = model.predict(imputed_X_valid)

#Metrics
print('Mean absolute error:', mean_absolute_error(y_valid,prediction))
print('Mean absolute percentage error:', mean_absolute_percentage_error(y_valid,prediction))
print('r2 score:',r2_score(y_valid,prediction))
print('root mean squared error:',root_mean_squared_error(y_valid,prediction))




Mean absolute error: 2978.5289330644564
Mean absolute percentage error: 0.3772056113113027
r2 score: 0.8239462963691347
root mean squared error: 5126.539718343397
